# Create a chart that shows mean city and highway mpg by make of the car

In [1]:
# standard import statements
import pandas as pd

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.io import show, output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
# REad data
auto = pd.read_excel('data/Auto_Insu.xlsx')
auto.columns

Index(['ID', 'Symboling', 'Normalized_Losses', 'Make', 'Fuel_Type',
       'Aspiration', 'NumDoors', 'Body_Style', 'Drive_Wheels',
       'Engine_Location', 'Wheel_Base', 'Length', 'Width', 'Height',
       'Curb_Weight', 'Engine_Type', 'NumCylinders', 'Engine_Size',
       'Fuel_System', 'Bore', 'Stroke', 'Compression_Ratio', 'HP', 'Peak_RPM',
       'City_MPG', 'Highway_MPG', 'Price'],
      dtype='object')

### Slicing DF to get average city and highway MPG by Make

In [6]:
mpg = auto.groupby('Make')['City_MPG','Highway_MPG','Price'].mean() 
# some older versions of bokeh do not support groupby object conversion to CDS, 
# USE #reset_index to convert back to DF in such cases.
# mpg = auto.groupby('Make')['City-MPG','Highway-MPG'].mean().reset_index() 

# converting groupby object to ColumnDataSource
cdsMPG = ColumnDataSource(mpg)

#Accessing names of column in ColumnDataSource
cdsMPG.column_names

['Make', 'City_MPG', 'Highway_MPG', 'Price']

## Add visual dodge effect to create clustered bar chart
See more on dodge at https://bokeh.pydata.org/en/latest/docs/user_guide/categorical.html#visual-dodge

dodge(field_name, value, range=None)
<br> Parameters:
<br><li> field_name (str) – a field name to configure DataSpec with
<br><li> value (float) – the fixed offset to add to column data
<br><li> range (Range, optional) – a range to use for computing synthetic coordinates when necessary, e.g. a FactorRange when column data is categorical (default: None)


In [7]:
from bokeh.transform import dodge

# Used for specifying legend value based on a variable
from bokeh.core.properties import value

from bokeh.models import HoverTool

In [13]:
# Specifying list of tuples to show the tooltips
# Adding formatting details in {} after the variable name.
# Variables are referred from columndatasource
myHover = HoverTool(tooltips=[("Average Price","@Price{$ 0.0 a}")])

In [16]:
# Create figure object
myAuto = figure(width=500, height=500,
               title="Mean city MPG by car make",
               x_axis_label='Make of the car',
               y_axis_label='Mean City MPG',
                # Always make sure categories are of type string and have unique values
               x_range=auto.Make.unique())

# Add vbar glyphs with dodge
myAuto.vbar(x=dodge('Make', 0, range=myAuto.x_range), color='red',
            top='City_MPG', width=.2, legend = value("City_MPG"), 
            muted_color='red', muted_alpha=.2,
            source=cdsMPG)

myAuto.vbar(x=dodge('Make', 0.25, range=myAuto.x_range), color='blue',
            top='Highway_MPG', width=.2, legend = value("Highway_MPG"), 
            muted_color='blue', muted_alpha=.2,
            source=cdsMPG)

# Add the hover tool
myAuto.add_tools(myHover)

# Adding interactive legends
#myAuto.legend.click_policy = 'hide'
# When adding mute policy, add muted_color and muted_alpha properties to glyph statements
myAuto.legend.click_policy = 'mute' 


myAuto.xaxis.major_label_orientation = 45 #"vertical" #45
show(myAuto)